In [1]:
%matplotlib widget
#%matplotlib inline

import os, sys

dir_path = os.path.abspath("")
sys.path.append(dir_path)
from workshop_utils import *

# Quantum Computing Simulation 

This notebook walks you through running a quantum circuit using several different common quantum computing frameworks: *PennyLane*, *Qiskit* and *CUDA-Q*. There are several others (e.g., *Cirq*, *Braket*), but we will focus on these three as they are popular and informative. These frameworks enable the noisy simulation of quantum circuits, though each has their own strengths and weaknesses. In this example, we will focus on the classical computational aspect of quantum computing simulation and how well different frameworks run and scale. 

We will walk you through the simulation of a quantum algorithm and provide Python and SLURM submission scripts to simulate it on HPC infrastructure without requiring running a Jupyter server on the HPC infrastructure. 



## The Computational Challenge

There is a real challenge in simulating quantum circuits. They require sufficient memory to store the possible combinations of qubit states and must perform many matrix-vector multiplications. A state-vector representation of a quantum circuit has a memory requirement that scales as $\mathcal{O}(2^{n})$, where $n$ is the number of qubits. This challenge is even more significant for a full density-matrix representation, which is  often used as an approach to simulate a noisy quantum computing device, as it scales as $\mathcal{O}(2^{2n})$. 

Consider running a full state-vector simulation with $n$-qubits and $m$ gates. Just the state vector requires $2\times2^{n}$ floats (since we need to store complex numbers). To store in full, $m$ arbitrary unitaries for a $n$-qubit system requries $2m\times2^{2n}$ floats and each gate is a matrix-vector operation, giving $\approx m\times2^{4n}$ floating point operations (for a nice discussion, see [https://arxiv.org/pdf/2302.08880]).

To explore the cost of full state simulations, explore the table below ([https://arxiv.org/pdf/2302.08880]).

#### Table I. Comparison between different simulation methods

| Methods         | Memory          | Run time          | Approx. or exact | Noiseless or noisy | Application regime |
|---------------|----------------|-------------------|------------------|--------------------|-------------------|
| **Statevector** | Worst $\mathcal{O}(2^N)$   | Worst $\mathcal{O}(m2^N)$    | Exact            | Noiseless#        | General, good for small circuits* |
| **Density matrix** | Worst $\mathcal{O}(2^{2N})$ | Worst $\mathcal{O}(m2^{2N})$    | Exact            | Both|  General, good for small circuits+ |
| **MPS state/MPO** | $\mathcal{O}(N\chi^2)$         | $\mathcal{O}(N\chi^6)$             | Approx.          | Noisy | General, good for shallow circuits |
| **Tensor network** | On demand     | $\mathcal{O}(e^W)$            | Both             | Both               | General, good for shallow circuits |
| **Stabilizer** | $\mathcal{O}(e^{mT})$        | $\mathcal{O}(e^{mT})$          | Approx.          | Both               | Circuits dominated with Clifford gates, particularly in QEC |

#### Notes:
\#. **State-vector simulators** can also be used to simulate noisy circuits to get an approximate result with the Monte Carlo method.  
\*. Circuits with N > 32 with the state-vector simulator should generally run on an HPC server.  
\+. Circuits with N > 16 with the density matrix simulator should generally run on an HPC server.       

Consequently, it becomes necessary to accelerate computation. However, not all quantum circuits demand a full state-vector simulation, and alternative methods—such as tensor network contractions, Stabilizer simulation and density matrix simulation —can significantly reduce computational complexity. In this work, we focus exclusively on performing full state-vector simulations across different frameworks and measuring the resulting time to solution to have a high-level overview. 

In [8]:
interact(PlotSystemRequirements, 
         num_qubits = widgets.IntSlider(min=1, max=128, step=1, value=10), 
         num_gates = widgets.IntSlider(min=1, max=10000, step=1, value=100),
         num_measurements = widgets.IntSlider(min=1, max=10000, step=1, value=100),
        )

interactive(children=(IntSlider(value=10, description='num_qubits', max=128, min=1), IntSlider(value=100, desc…

<function workshop_utils.PlotSystemRequirements(num_qubits: int = 2, num_gates: int = 1, num_measurements: int = 1, returnfig: bool = False)>

## Quantum Computing Frameworks

A wide array of quantum computing frameworks is currently available, each with its own advantages, limitations, and user communities. In this workshop, we highlight three commonly utilized frameworks:

### PennyLane
[PennyLane](https://pennylane.ai/) is an open-source Python library for quantum programming. Originally developed by researchers, it offers extensive functionality tailored to machine learning applications, along with support for multi-core and GPU acceleration.

### Qiskit
[Qiskit](https://www.ibm.com/quantum/qiskit) is an open-source quantum computing framework widely recognized for its robust toolset for constructing and optimizing quantum circuits. It includes Qiskit Aer, a high-performance simulator featuring realistic noise models, with support for OpenMP, MPI, and GPU-based acceleration.

### CUDA-Q
[CUDA-Q](https://developer.nvidia.com/cuda-q) is an open-source framework, although it relies on certain closed-source libraries. It facilitates hybrid application development in quantum computing by providing a unified programming model that coordinates CPUs, GPUs, and QPUs. CUDA-Q supports both Python and C++, offering multiple backends optimized for multi-core or GPU acceleration.


## Grover's Algorithm for Quantum Search

First proposed by Lov Grover in 1996 [Grover, L. K. (1996, July). A fast quantum mechanical algorithm for database search. In Proceedings of the twenty-eighth annual ACM symposium on Theory of computing (pp. 212-219).], Grover's algorithm is a canonical example of quantum speedup. It addresses the problem of unstructured search, identifying with high probability the unique input to a black-box function that produces a specified output. By reducing the number of required function evaluations to $\mathcal{O}(\sqrt{N})$, where $N$ represents the size of the function's domain, Grover’s algorithm offers a quadratic advantage over the classical approach, which necessitates $\mathcal{O}(\sqrt{N})$ queries. Grover's algorithm revolutionized quantum computing by demonstrating a quadratic speedup for unstructured search problems, showcasing the power of quantum amplitude amplification.

### Manually Finding a Target

To illustrate this concept, consider a database containing various entries, with no prior information to guide your search. The only option is to check entries individually to determine if one matches your target. This scenario can be likened to a game of “Where’s the ball?”—you systematically check possible locations until you find the correct one.

Below, you will see a 4×4 grid. Click on the circles to discover the hidden target. Try this process a few times, then restart the cell to repeat the exercise.

In [ ]:
# now this is a manual search of a grid  
fig = GroversGrid(4,4)
fig 

### Quantum approach

Compared with the manually finding a target, in this quantum approach, the probability of projecting onto the target state is enhanced by applying an appropriately chosen phase shift. By adjusting the slider, one can visualize how varying the phase difference between the target state and all other states affects the system’s evolution. Systematic exploration reveals that, at an optimal phase setting, the probability (and corresponding visual representation) of the target state becomes significantly larger than that of the other states. Notably, determining the maximal probability of success requires identifying not only the ideal phase shift but also the optimal measurement time, as both parameters jointly influence the outcome.


In [ ]:
fig = GroverSearch(4, 4)
fig 

## So how do we achieve this? 
### Step 1 Uniform Superposition
The first step in Grover's search is generating a *uniform superposition* (i.e. a quantum state in which each computational basis state appears with equal amplitude in magnitude, though possibly differing in phase, ensuring that measurement probabilities remain uniform). By convention, all every qubit is initialised in the $\ket{0}$ state, so a uniform superposition is obtained by applying a *Hadamard* ($H$) gates to each qubit. A $H$ gate acts on the $\ket{0}$ state like:

$$
H\ket{0} = \frac{\ket{0} + \ket{1}}{\sqrt{2}}.
$$

In terms of circuits, a 3-qubit circuit that has all the qubits in superposition will look like:

![3 Hadamard](figures/hadamard_example.png "3 Qubit Hadamard circuit")

#### Frameworks 

Let's start by first going through some syntax. In most frameworks, you define a circuit that will run Grover's search. For completeness, we show the API for all three frameworks here explicitly. In later steps, for brevity, we'll use functions defined in `workshop_utils.py`. 

* *Pennylane*:
    ```python
    import pennylane as qml
    # Define a quantum device (real or simulator)
    dev = qml.device("default.qubit", wires=n)
    # Define a quantum function (QNode) that modifies the quantum circuit on device `dev`
    @qml.qnode(dev)
    def circuit(n : int):
        # Create n wires for n qubits
        wires = list(range(n))
        # Place a H gate on each wire
        for w in wires:
            qml.Hadamard(wires=w)
    ```
* *Qiskit*:
  ```python
  from qiskit import QuantumCircuit
  def circuit(n : int) -> QuantumCircuit:
      # Initialise a quantum circuit with n qubits
      qc = QuantumCircuit(n)
      # Apply a H gate to each qubit
      for i in range(n):
          qc.h(i)
      # Return the modified circuit
      return qc
  ```

* *CUDA-Q*:
  ```c++
  // C++ API
  #include <cudaq.h>
  #include <cudaq/spin_op.h>
  #include <cudaq/algorithms/draw.h>
  // Define a quantum kernel (function)
  struct kernel
  {
      auto operator()(int n) __qpu__
      {
          // Create a quantum register (vector of qubits) with n qubits
          auto wires = cudaq::qvector(n);
          // Apply a Hadmard gate to each qubit
          cudaq::h(wires);
      }
  }
  ```
  ```python
  # Python API
  import cudaq
  # Define a quantum kernel (function)
  @cudaq.kernel
  def circuit(n : int):
      # Create a quantum register (vector of qubits) with n qubits
      qvector = cudaq.qvector(n)
      # Apply a Hadmard gate to each qubit
      h(qvector)
  ```

### Step 2 Amplification 

The next step is to apply rounds of amplification to increase the probability of finding the desired target state $\ket{\omega}$. This involves formulating a unitary operator that flips the phase of the target state while keeping all other states untouched: 

$$
U_\omega \ket{x} =
\begin{cases} 
-\ket{x}, & \text{if } x = \omega, \\
\,\,\,\,\,\ket{x}, & \text{if } x \neq \omega.
\end{cases}
$$
Which is explicitly given by: 
$$
U_\omega=\mathbb{I}−2\ket{\omega}\bra{\omega}.
$$


Since this transformation only modifies the phase of the target state, it does not affect the measurement probabilities. However, in the next step, we amplify the phase-marked target state using the Grover diffusion operator, which effectively performs an inversion about the mean by reflecting the state across the uniform superposition $\ket{s}$. This step enhances the probability amplitude of the target state, increasing its likelihood of being measured:

$$
U_d=2\ket{s}\bra{s}-\mathbb{I} .
$$

The combination of $U_\omega$ with $U_d$ rotates the state $\ket{s}$ by an angle of $\theta=2\arcsin\bigl(\tfrac{1}{\sqrt{N}}\bigr)$. 

The resulting circuit for two rounds of amplification using the PennyLane API for the two unitaries would look like 

![Grovers](figures/grovers_2_amplification.png "3 Qubit Grover's circuit with 2 amplification rounds")


#### Frameworks

* *PennyLane*: there is the `FlipSign` function for invoking $U_\omega$ and `GroverOperator` for $U_d$. ,
* *Qiskit*: construct the Oracle operator that marks the state and then pass this to the `GroverOperator` fucntion which incorporates both unitaries.
* *CUDA-Q*: both unitaries are done with explicit application of the appropriate gates, such as X-gates, Z-gates, Hadamards, and control gates, i.e.:
  
  ```c++
  cudaq::h(qvector);
  cudaq::x(qvector);
  cudaq::control(ApplyZ{}, qvector.front(n - 1), qvector.back());
  ```

## Let's Simulate!

Let's try running Grovers in *PennyLane*, *Qiskit* and *CUDA-Q*.

In [2]:
w = interactive(GroversPennyLane,
         num_qubits = widgets.IntSlider(min=2, max=20, step=1, value=10), 
         num_amplification = widgets.IntSlider(min=1, max=10, step=1, value=1),
         num_shots = widgets.IntSlider(min=100, max=1000, step=1, value=100),
         num_iterations = widgets.IntSlider(min=1, max=100, step=1, value=1),
         targets = fixed([]), 
         verbose = fixed(True),
         get_return = fixed(False),
         report_system_requirements=fixed(False),
        )
display(w)

interactive(children=(Text(value='default.qubit', description='device'), IntSlider(value=10, description='num_…

In [3]:
w = interactive(GroversQiskit,
         num_qubits = widgets.IntSlider(min=2, max=20, step=1, value=10), 
         num_amplification = widgets.IntSlider(min=1, max=10, step=1, value=1),
         num_shots = widgets.IntSlider(min=100, max=1000, step=1, value=100),
         num_iterations = widgets.IntSlider(min=1, max=100, step=1, value=1),
         targets = fixed([]), 
         verbose = fixed(True),
         get_return = fixed(False),
         report_system_requirements=fixed(False),
        )
display(w)

interactive(children=(Text(value='CPU', description='device'), IntSlider(value=10, description='num_qubits', m…

In [ ]:
w = interactive(GroversCUDAQ,
         num_qubits = widgets.IntSlider(min=2, max=20, step=1, value=10), 
         num_amplification = widgets.IntSlider(min=1, max=10, step=1, value=1),
         num_shots = widgets.IntSlider(min=100, max=1000, step=1, value=100),
         num_iterations = widgets.IntSlider(min=1, max=100, step=1, value=1),
         targets = fixed([]), 
         verbose = fixed(True),
         get_return = fixed(False),
         report_system_requirements=fixed(False),
        )
display(w)
# [circuit, time, err, tottime] = GroversCUDAQ(device = 'nvidia', num_qubits=13)

## Scaling

We can explore how well codes scale and the variability. 

In [ ]:
# lets get the data 
maxnqubits=10
qubits = np.arange(3, maxnqubits+1, dtype=np.int32)

times = {
    'PennyLane': {
        'default.qubit': np.zeros([3,qubits.size]),
    },
    'Qiskit': {
        'CPU': np.zeros([3,qubits.size]),
    },
    'CUDAQ': {
        'CPU': np.zeros([3,qubits.size]),
        'GPU': np.zeros([3,qubits.size]),
    },
    'Ideal': {
        'FLOPS': np.zeros([3,qubits.size]),
    },
}

funcs = {
    'PennyLane': GroversPennyLane,
    'Qiskit': GroversQiskit,
    'CUDAQ': GroversCUDAQ,
}

for api in funcs.keys():
    for device in times[api].keys():
        for i in range(qubits.size):
            # for skipping CUDAQ on systems lacking CUDAQ 
            if api == 'CUDAQ': continue
            [circuit, times[api][device][0][i], times[api][device][1][i], times[api][device][2][i]] = \
            funcs[api](device = device, num_qubits = int(qubits[i]), num_amplification = 10, verbose = False)

times['Ideal']['FLOPS'][0] = np.exp(flopcalc(qubits, qubits + 10*(qubits*4 + (qubits-1)*2), 1))
print(times['Ideal']['FLOPS'][0])


In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols = 2, nrows = 1, figsize=(10, 5))
# standard error bars
for api in funcs.keys():
    for device in times[api].keys():
        ax1.errorbar(qubits, times[api][device][0], yerr=times[api][device][1], 
                    linestyle='solid', 
                    label = f'{api}:{device}',
                    marker = 'o',
                    capsize= 4,
                   )
# standard error bars
for api in funcs.keys():
    for device in times[api].keys():
        ax2.plot(qubits, times[api][device][2],
                    linestyle='solid', 
                    label = f'{api}:{device}',
                    marker = 'o',
                   )

ax1.legend()
ax1.set_ylabel('Time per shot [s]')
ax1.set_xlabel('Number of qubits')
ax1.set_yscale('log')
ax2.set_ylabel('Total Time [s]')
ax2.set_xlabel('Number of qubits')
ax2.set_yscale('log')

fig

### GraceHopper Scaling Results 

A comparative analysis of *PennyLane*, *Qiskit*, and *CUDA-Q* in full state-vector simulations of Grover’s search with ideal qubits highlights how different frameworks impose practical constraints on quantum algorithm exploration. *PennyLane*, while offering a versatile set of features, can suffer from inefficient GPU resource utilization, leading to performance bottlenecks in large-scale simulations. *Qiskit*, on the other hand, encounters challenges related to unified memory access on the GH200 platform, introducing significant overhead costs that can limit scalability. In contrast, *CUDA-Q* , though more specialized and with fewer high-level capabilities, demonstrates superior efficiency in executing specific quantum algorithms. Its ability to leverage optimized memory management and GPU acceleration enables the simulation of larger qubit systems on a single node, making it particularly advantageous for high-performance quantum computing workloads.

![4 Scaling](figures/grovers_scaling_example.png "4 Scaling of Grovers Circuit")
